<a href="https://colab.research.google.com/github/sts3117/cucumberProject/blob/main/0213_ourmodel_me_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import PIL  # python 의 이미지 전처리 라이브러리
import glob # 대용량, 대규모 파일을 다룰때 사용
import os # os, path 관련 조작, 변경 , 파일 삭제 등
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

Mounted at /content/drive


In [2]:
# /content/drive/MyDrive/dataset.zip
!mkdir dataset
!mkdir dataset01


In [ ]:
!unzip /content/drive/MyDrive/dataset.zip -d dataset

Archive:  /content/drive/MyDrive/dataset.zip
replace dataset/cucumber/cucumber_fifth-001.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: unzip:  cannot find or open /content/drive/MyDrive/dataset01.zip, /content/drive/MyDrive/dataset01.zip.zip or /content/drive/MyDrive/dataset01.zip.ZIP.


In [ ]:
!unzip /content/drive/MyDrive/dataset01.zip -d dataset01

In [ ]:
dataset_folder_path = '/content/dataset'
category_cnt = os.listdir(dataset_folder_path)
print(os.listdir(dataset_folder_path))

[]


In [4]:
dataset_folder_path = '/content/dataset01'
category_cnt = os.listdir(dataset_folder_path)
print(os.listdir(dataset_folder_path))

['cucumber01', 'zucchini01']


In [5]:
# glob.glob(dataset_folder_path+'/*/*')
# 파일의 경로+/* 을 입력하면 해당 폴더 하위 경로 상에 있는 모든 파일을 리스트로 리턴
# ex) dataset_folder_path+'/*/*' -> 하위 폴더의 하위 데이터 전체 리턴

x = []
y = []

for label, folder in enumerate(os.listdir(dataset_folder_path)):
    # print(dataset_folder_path +'/'+ folder) # 좋지 코드가 아님
    # os.path.join(dataset_folder_path, folder) # 유지 보수 좋은 코드
    folder_full_path = os.path.join(dataset_folder_path, folder)
    all_files = glob.glob(folder_full_path + '/*.jpeg')
    all_files_jpg = glob.glob(folder_full_path + '/*.jpg')
    all_files_png = glob.glob(folder_full_path + '/*.png')
    all_files.extend(all_files_jpg)
    all_files.extend(all_files_png)

    for idx, file_full_path in enumerate(all_files):
        image = PIL.Image.open(file_full_path)
        image = image.resize((64, 64))
        image = image.convert('RGB')
        data = np.asarray(image)
        x.append(data)
        y.append(label)
        if idx % 30 == 0:
            print(idx, '/', len(all_files))

0 / 588
30 / 588
60 / 588
90 / 588
120 / 588
150 / 588
180 / 588
210 / 588
240 / 588
270 / 588
300 / 588
330 / 588
360 / 588
390 / 588
420 / 588
450 / 588
480 / 588
510 / 588
540 / 588
570 / 588
0 / 284
30 / 284
60 / 284
90 / 284
120 / 284
150 / 284
180 / 284
210 / 284
240 / 284
270 / 284


In [6]:
x = np.array(x)
y = np.array(y)
print('카테고리 갯수', np.bincount(y))
print('전처리 확인', x.shape)

카테고리 갯수 [588 284]
전처리 확인 (872, 64, 64, 3)


In [7]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
# # 개발 팁, 전처리 시간이 너무 오래 걸린다면(데이터 늘어 나서)
# xy = (x_train, x_test, y_train, y_test)
# np.save('/content/drive/MyDrive/preprocessing_data.npy', xy)
# x_train, x_test, y_train, y_test = np.load('/content/drive/MyDrive/preprocessing_data.npy')

In [8]:
x_train = x_train / 255.
x_test = x_test / 255.

In [ ]:
# 구글넷 실습(오전에 진행한)
# 레즈넷 실습(다 짜여진 코드) -> fine tuning

In [3]:

# 개선의 여지 해상도 수정 ( 전처리 도 수정 해야함)
input_layer = layers.Input(shape = (64,64,3), name='start_layer')

# 개선 의 여지 conv2d 수정 (성능이 매우 잘 나온다면  - 모델 다이어트
# 성능이 안나온다면 channel 수, 레이어 추가
tower_1 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)

tower_2 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_2 = layers.Conv2D(64, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2)

tower_3 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_3 = layers.Conv2D(64, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3)

tower_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(input_layer)
tower_4 = layers.Conv2D(64, kernel_size=(1,1) , padding='same' , activation='relu')(tower_4)

concat_layer = layers.concatenate([tower_1, tower_2,tower_3,tower_4], axis=3)

tower_2_1 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)

tower_2_2 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_2 = layers.Conv2D(120, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2_2)

tower_2_3 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_3 = layers.Conv2D(120, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_2_3)

tower_2_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer)
tower_2_4 = layers.Conv2D(120, kernel_size=(1,1) , padding='same' , activation='relu')(tower_2_4)

final_concat_layer = layers.concatenate([tower_2_1, tower_2_2,tower_2_3,tower_2_4], axis=3)
average_pooling_layer = layers.AveragePooling2D(pool_size=(16,16),strides=(16,16))(final_concat_layer)
flat_layer = layers.Flatten()(average_pooling_layer)
# 개선 의 여지 -  Dense 층 추가
output_layer = layers.Dense(16, activation = 'relu')(flat_layer)
output_layer = layers.Dense(1, activation = 'sigmoid')(output_layer)

google_net_model = models.Model(inputs = input_layer, outputs = output_layer)
google_net_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start_layer (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_1 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 conv2d_3 (Conv2D)           (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 64, 64, 3)            0         ['start_layer[0][0]']         
 D)                                                                                           

In [10]:
# 개선 여지 hyper parameter tuning
# ex) tf.keras.optimizers.Adam()
google_net_model.compile(loss='binary_crossentropy',
                         optimizer='adam', metrics=['accuracy'])

In [11]:
check_point_path = '/content/drive/MyDrive/ourmodel_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path, monitor='val_loss', save_best_only=True)
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)

In [12]:
google_net_model.fit(x_train, y_train, batch_size=32,
                     epochs=20,
                     callbacks=[mc, es],

                     validation_data=(x_test, y_test))

Epoch 1/20
22/22 [==============================] - 27s 665ms/step - loss: 0.6579 - accuracy: 0.6628 - val_loss: 0.6099 - val_accuracy: 0.7143
Epoch 2/20
22/22 [==============================] - 3s 128ms/step - loss: 0.6495 - accuracy: 0.6643 - val_loss: 0.6148 - val_accuracy: 0.7143
Epoch 3/20
22/22 [==============================] - 5s 224ms/step - loss: 0.6365 - accuracy: 0.6643 - val_loss: 0.6026 - val_accuracy: 0.7143
Epoch 4/20
22/22 [==============================] - 5s 243ms/step - loss: 0.6205 - accuracy: 0.6743 - val_loss: 0.5907 - val_accuracy: 0.7143
Epoch 5/20
22/22 [==============================] - 3s 129ms/step - loss: 0.6154 - accuracy: 0.6815 - val_loss: 0.5936 - val_accuracy: 0.6971
Epoch 6/20
22/22 [==============================] - 3s 129ms/step - loss: 0.6055 - accuracy: 0.6829 - val_loss: 0.6309 - val_accuracy: 0.7143
Epoch 7/20
22/22 [==============================] - 3s 130ms/step - loss: 0.6150 - accuracy: 0.6872 - val_loss: 0.6028 - val_accuracy: 0.7086
Epoch

In [14]:
## our model inference code
# inference만 진행 할 시, 가장 위 import library 실행 -> 모델 빌드 하는 부분 실행
google_net_model.load_weights('/content/drive/MyDrive/ourmodel_checkpoint')

In [15]:
from google.colab import files
f = files.upload()

Saving cucumber01-367.jpeg to cucumber01-367.jpeg


In [16]:
file_name = list(f.keys())[0]
test_img = PIL.Image.open(file_name)
test_img = test_img.resize((64, 64))
test_img = test_img.convert('RGB')
data = np.asarray(test_img)
test_image_tensor = np.array([data])
test_image_tensor = test_image_tensor / 255.
test_image_tensor.shape


(1, 64, 64, 3)

In [17]:
result = google_net_model.predict(test_image_tensor)
result[0][0]
if result[0][0] > 0.5:
    print(f'{result[0][0] * 100} % 확률로 cucumber 로 판단이 됩니다.')
else:
    print(f'{1 - result[0][0] * 100} % 확률로 zucchini 로 판단이 됩니다.')

1/1 [==============================] - 0s 131ms/step
-24.6646990776062 % 확률로 zucchini 로 판단이 됩니다.


성능 개서 ( 모델 수정, 데이터, 하이퍼메터 튜닝)

목표 80% 현재 성능 70%
1. 데이터
2. 모델
3. 하이퍼 파라메터 수정 (우선 순위 최하) : 성능 수치 79.7% 일때